In [12]:
import altair as alt
import pandas as pd
import numpy as np
import re


In [13]:
airport_url = 'csv/airports.csv'
airports_df = pd.read_csv(airport_url, encoding='latin-1')


In [14]:
airports_df.head(5)


,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,TimeZone,DST,Tz Database Time Zone,Type,Source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789002,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


### Derive country continents from new dataset

In [15]:
# number of airports in each continent
# find out how to derive continent
continents = pd.read_csv('csv/continents.csv')
continents
# drop code and year attributes
continents.drop(columns=['Code', 'Year'],inplace=True)
continents
# aggregate all airports in each continent

,Entity,Continent
0,Abkhazia,Asia
1,Afghanistan,Asia
2,Akrotiri and Dhekelia,Asia
3,Albania,Europe
4,Algeria,Africa
...,...,...
280,Yugoslavia,Europe
281,Zambia,Africa
282,Zanzibar,Africa
283,Zimbabwe,Africa


Certain country names do not match in the datasets so they must be normalized. To do this we must create a copy of the original dataframe and replace the unmatching countries with their continents. we end up with a mix of countries and continents in one column

In [18]:

# # for each row in airports, check if country is equal to entity (continents), append continent to it
from pandas import DataFrame


temp_df = airports_df.copy()
for country in temp_df['Country'].to_list():
    for row in continents.values:
        if country.lower() in row[0].lower():
            temp_df['Country'] = temp_df['Country'].replace(
                country, row[1])
            break
mixed_continents_and_countries = temp_df['Country'].unique()


def normalize_country_names(df_to_be_normalized, reference_df):
    # # for each row in df_to_be_normalized, check if country is equal to country in reference df
    temp_df = df_to_be_normalized.copy()
    for country in temp_df['Country'].to_list():
        for row in reference_df.values:
            if country.lower() in row[0].lower():
                temp_df['Country'] = temp_df['Country'].replace(
                    country, row[1])
                break
    mixed_continents_and_countries = temp_df['Country'].unique()
    
    # Next we must extract these countries into their own list
    unmatched_countries = []
    for country in mixed_continents_and_countries:
        if country.lower() not in [x.lower() for x in reference_df['Continent'].to_list()]:
            unmatched_countries.append(country)
    
    
    # Finally, we create a copy of the original dataframe and replace the unmatched country names   
    new_country_names = df_to_be_normalized.copy()

    for country in df_to_be_normalized['Country'].to_list():
        for x in unmatched_countries:
            if x.lower() == country.lower():
                new_country_names['Country'] = new_country_names['Country'].replace(x)
    
    # Now we can finally add continent information to the dataset     
    continents_list = []
    for country in new_country_names['Country'].to_list():
        for row in continents.values:
            if country.lower() in row[0].lower():
                continents_list.append(row[1])
                break
    new_country_names['Continent'] = continents_list
    df_to_be_normalized = new_country_names
    # export to new csv
    df_to_be_normalized.to_csv(f'csv/{df_to_be_normalized}-1.csv',index=False)


In [26]:
def normalize_country_names(df_to_be_normalized:DataFrame,reference_df:DataFrame,
                            reference_df_country_column_name:str, reference_df_column:str
                           ):
    # copy df_to_be_normalized
    temp_df = df_to_be_normalized.copy()
    # replace unmatching countries with the column you want to add
    for country in temp_df['Country'].to_list():
        for index,row in reference_df.iterrows():
            if country.lower() in row[reference_df_country_column_name].lower():
                temp_df['Country'] = temp_df['Country'].replace(
                    country, row[1])
                break
    mixed_country_row = temp_df['Country'].unique()
    
    
    unmatched_countries = []
    for country in mixed_continents_and_countries:
        if country.lower() not in [x.lower() for x in reference_df['Continent'].to_list()]:
            unmatched_countries.append(country)

normalize_country_names(airports_df,continents,'Entity')

array(['Oceania', 'North America', 'Europe', 'Africa', 'South America',
       'Faroe Islands', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Swaziland', 'Asia', 'Czech Republic', 'Antarctica', 'West Bank',
       'Midway Islands', 'Macau', 'Burma', 'East Timor', 'Johnston Atoll',
       'Cocos (Keeling) Islands', 'Wake Island'], dtype=object)

In [ ]:
mixed_continents_and_countries

array(['Oceania', 'North America', 'Europe', 'Africa', 'South America',
       'Faroe Islands', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Swaziland', 'Asia', 'Czech Republic', 'Antarctica', 'West Bank',
       'Midway Islands', 'Macau', 'Burma', 'East Timor', 'Johnston Atoll',
       'Cocos (Keeling) Islands', 'Wake Island'], dtype=object)

Next we must extract these countries into their own list

In [ ]:
unmatched_countries = []
for country in mixed_continents_and_countries:
    if country.lower() not in [x.lower() for x in continents['Continent'].to_list()]:
        unmatched_countries.append(country)
            
unmatched_countries

['Faroe Islands',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Swaziland',
 'Czech Republic',
 'West Bank',
 'Midway Islands',
 'Macau',
 'Burma',
 'East Timor',
 'Johnston Atoll',
 'Cocos (Keeling) Islands',
 'Wake Island']

Finally, we create a copy of the original dataframe and replace the unmatched country names

In [ ]:
new_country_names = airports_df.copy()

for country in airports_df['Country'].to_list():
    for x in unmatched_countries:
        if x.lower() == country.lower():
            new_country_names['Country'] = new_country_names['Country'].replace(x)

Now we can finally add continent information to the dataset

In [ ]:
continents_list = []
for country in new_country_names['Country'].to_list():
    for row in continents.values:
        if country.lower() in row[0].lower():
            continents_list.append(row[1])
            break
new_country_names['Continent'] = continents_list
airports_df = new_country_names
# export to new csv
airports_df.to_csv('csv/airports_continents.csv',index=False)